| ![EEW logo](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/eew.jpg?raw=true) | ![EDGI logo](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/edgi.png?raw=true) |
|---|---|

#### This notebook is licensed under GPL 3.0. Please visit our Github repo for more information: https://github.com/edgi-govdata-archiving/ECHO-Cross-Program
#### The notebook was collaboratively authored by EDGI and PInT from Olin College following our authorship protocol: https://docs.google.com/document/d/1CtDN5ZZ4Zv70fHiBTmWkDJ9mswEipX6eCYrwicP66Xw/ PInT Contributors: Olivia Chang, Trevor Zou, Lili Baker

#### For more information about this project, visit https://www.environmentalenforcementwatch.org/

---

## How to Run
* A "cell" in a Jupyter notebook is a block of code performing a set of actions making available or using specific data.  The notebook works by running one cell after another, as the notebook user selects offered options.
* If you click on a gray **code** cell, a little “play button” arrow appears on the left. If you click the play button, it will run the code in that cell (“**running** a cell”). The button will animate. When the animation stops, the cell has finished running.
![Where to click to run the cell](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/pressplay.JPG?raw=true)
* You may get a warning that the notebook was not authored by Google. We know, we authored them! It’s okay. Click “Run Anyway” to continue. 
![Error Message](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/warning-message.JPG?raw=true)
* **It is important to run cells in order because they depend on each other.**
* Run all of the cells in a Notebook to make a complete report. Please feel free to look at and **learn about each result as you create it**!

---

# **Let's begin!**

Hover over the "[ ]" on the top left corner of the cell below and you should see a "play" button appear. Click on it to run the cell then move to the next one.

These first two cells give us access to some external Python code we will need.

## Bring in some code that is stored in a Github project.
These two github repositories, geopandas, and rtree hold Python code that the notebook uses. These are useful tools for interpreting data.
* ECHO_modules holds code that can be used in this and other notebooks


In [ ]:
!git clone https://github.com/edgi-govdata-archiving/ECHO_modules.git &>/dev/null;
# !git clone https://github.com/oliviachang29/ECHO_modules.git &>/dev/null;
!pip install geopandas &>/dev/null;
!pip install rtree &>/dev/null;
!pip install selenium &>/dev/null;
!pip install scikit-learn &>/dev/null;

print("Done!")

## Run a few Python modules.
These will help us process and visualize the different program data sets later. It is not important to know what the python modules are as they are only used to process and visualize the data.

In [ ]:
from ECHO_modules.get_data import get_echo_data
from ECHO_modules.utilities import show_region_type_widget, \
    show_state_widget, show_pick_region_widget, get_active_facilities, \
    state_choropleth_mapper

import pandas as pd
import math
import geopandas
import rtree
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import pandas as pd
import requests
from xml.etree import ElementTree
import numpy as np
import folium
import json
from functools import reduce
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
print("Done!")

## Get facilities data from the database.


Stony Brook University that regularly scrapes the information from the [EPA's ECHO website](echo.epa.gov/) and loads it into a database format that we can use to run research. The cell below retrieves all of the facilities in ECHO and stores it in the variable `full_echo_data`.

The sql query determines which columns from the database are loaded. Adding more columns will make this cell take longer to run. As is, this cell may take several minutes to load because it is retrieving every facility (row) in the data. That's over 1 million records!

**Want to learn more about what each column means?**
[This spreadsheet catalogs the meaning of each column.](https://docs.google.com/spreadsheets/d/1LB-dlkwss60UNN_xjtCbU6TLhiPPls3ResCVpNOSBuI/edit?usp=sharing)

In [ ]:
sql = 'select  "REGISTRY_ID","FAC_NAME","FAC_DERIVED_CB2010","EJSCREEN_FLAG_US","FAC_PERCENT_MINORITY","FAC_LONG", "FAC_LAT", "FAC_STATE","FAC_COUNTY","FAC_CITY","FAC_ZIP","FAC_DERIVED_CD113","FAC_EPA_REGION","NPDES_IDS","NPDES_FLAG","CWA_INSPECTION_COUNT","CWA_DAYS_LAST_INSPECTION","CWA_INFORMAL_COUNT","CWA_FORMAL_ACTION_COUNT","CWA_PENALTIES","CWA_LAST_PENALTY_AMT","CWA_QTRS_WITH_NC","RCRA_IDS","RCRA_FLAG","RCRA_INSPECTION_COUNT","RCRA_DAYS_LAST_EVALUATION","RCRA_INFORMAL_COUNT","RCRA_FORMAL_ACTION_COUNT","RCRA_PENALTIES","RCRA_LAST_PENALTY_AMT","RCRA_QTRS_WITH_NC","AIR_IDS","AIR_FLAG","CAA_EVALUATION_COUNT","CAA_DAYS_LAST_EVALUATION","CAA_INFORMAL_COUNT", "CAA_FORMAL_ACTION_COUNT","CAA_PENALTIES","CAA_LAST_PENALTY_AMT","CAA_QTRS_WITH_NC" from "ECHO_EXPORTER" where "FAC_ACTIVE_FLAG" = \'Y\''

try:
# Don't index.
  full_echo_data = get_echo_data( sql )
except EmptyDataError:
  print("\nThere are no records.\n")
  
full_echo_data

### Assumptions

As you can see, many facilities have "NaN", or "Not a Number", listed in a column. We make the assumption that NaN means missing data, not 0, for columns like CWA_INSPECTION_COUNT. This can be confirmed by downloading EPA's data for yourself from [here](https://echo.epa.gov/files/echodownloads/echo_exporter.zip). You will see that EPA has left many cells blank. There are good reasons for this: EPA itself may not know, for instance, how many inspections have been done for a facility in the past 3-5 years. Rather than imputing a potentially false 0, they seem to have decided to leave such values blank. 

## Sort facilities by Clean Water Act (CWA), Resource Conservation and Recovery Act (RCRA), and Clean Air Act (CAA).
These cells get all facilities in the database that are marked as regulated under the CWA, RCRA, and CAA respectively, and are currently active.

Some facilities can be marked as regulated under multiple acts, so some facilities will be contained in two or more of the CWA, RCRA, and CAA reports we generate.

Each table below tabulates this data, but only displays the first and last five rows.

#### **Clean Water Act (CWA) Facilities**
For each facility, if `NPDES_FLAG` is `Y`, then the facility must report under the CWA National Pollutant Discharge Elimination System (NPDES). `cwa_facs`, shown in this table, contains all the facilities that must report under the CWA by filtering facilities by `NPDES_FLAG`. 

In [ ]:
cwa_facs = full_echo_data[full_echo_data["NPDES_FLAG"] == "Y"]
cwa_facs[["FAC_NAME", "NPDES_FLAG"]]

#### **RCRA Facilities**
For each facility, if `RCRA_FLAG` is `Y`, then the facility must report under the RCRA. `rcra_facs`, shown in this table, contains all the facilities that must report under the RCRA by filtering facilities by `RCRA_FLAG`.

In [ ]:
rcra_facs = full_echo_data[full_echo_data["RCRA_FLAG"] == "Y"]
rcra_facs[["FAC_NAME", "RCRA_FLAG"]]

#### **CAA Facilities**
For each facility, if `AIR_FLAG` is `Y`, then the facility must report under the RCRA. `caa_facs`, shown in this table, contains all the facilities that must report under the CAA by filtering facilities by `AIR_FLAG`.

In [ ]:
caa_facs = full_echo_data[full_echo_data["AIR_FLAG"] == "Y"]
caa_facs[["FAC_NAME", "AIR_FLAG"]]

Now, we'll put `cwa_facs`, `rcra_facs`, and `caa_facs` into a format that makes it more accessible for the following cells.

In [ ]:
# First, we name the fields, or columns, that are relevant to each regulatory program.
# In addition to basic info pertient for any facility, we are interested in...
# things like the days since a facility's last inspection, the number of informal regulatory actions taken against it,
# the number of formal actions taken against it, $ value of penalties, and time spent in non-compliance.

intended_columns = {
  "Basic Info": ["REGISTRY_ID", "FAC_LONG", "FAC_LAT", "FAC_STATE", "FAC_COUNTY"],
  "CWA": ["CWA_INSPECTION_COUNT", "CWA_DAYS_LAST_INSPECTION", "CWA_INFORMAL_COUNT", "CWA_FORMAL_ACTION_COUNT", "CWA_PENALTIES", "CWA_LAST_PENALTY_AMT", "CWA_QTRS_WITH_NC"],
  "RCRA": ["RCRA_INSPECTION_COUNT","RCRA_DAYS_LAST_EVALUATION","RCRA_INFORMAL_COUNT","RCRA_FORMAL_ACTION_COUNT","RCRA_PENALTIES","RCRA_LAST_PENALTY_AMT","RCRA_QTRS_WITH_NC"],
  "CAA": ["CAA_EVALUATION_COUNT","CAA_DAYS_LAST_EVALUATION","CAA_INFORMAL_COUNT", "CAA_FORMAL_ACTION_COUNT","CAA_PENALTIES","CAA_LAST_PENALTY_AMT","CAA_QTRS_WITH_NC"]
}

act_facs = {
  "Basic Info": full_echo_data,
  "CWA": cwa_facs,
  "RCRA": rcra_facs,
  "CAA": caa_facs
}

# Exploring Missing Data

This function counts the number of facilities in `full_echo_data` that are missing information in a column. This cell won't generate results, but run it in order to proceed with the analysis.

In [ ]:
def print_num_missing_per_column(column):
  print("# Facilities missing " + column + ":")
  count = full_echo_data[full_echo_data[column].isnull()].shape[0]
  pct = (count / full_echo_data.shape[0]) * 100
  # print number with commas
  print(f'{count:,}', "facilities")
  print(f'{pct:.2f}', "%")
  print("-------------------------")
print("Ready!")

### Many facilities are missing basic data, like location, registry ID, and name

If these columns are missing, regulators are unable to complete basic regulatory tasks. Run this cell to see a breakdown of the number and percent of facilities missing basic data.

In [ ]:
for column in intended_columns["Basic Info"]:
  print_num_missing_per_column(column)

### Some facilities are missing data that we need to conduct environmental justice research:

These two columns help us determine whether a regulated facility is situated in a majority-minority community:

* `FAC_PERCENT_MINORITY` is EPA's estimate of the percentage of the population within a 3-mile radius of the facility that is minority (non-white).
* `FAC_DERIVED_CB2010` is the 2010 Census Block where the facility is located. Without this, we can't compare the CB2010 block number to census data to get information on the facility's surrounding community.

Additionally, `EJSCREEN_FLAG_US` indicates whether a facility is located in a census block groups that is in the 80th or higher national percentile of one of the primary environmental justice indexes of EJSCREEN, EPA's screening tool for EJ concerns.


In [ ]:
ej_columns = ["FAC_PERCENT_MINORITY", "FAC_DERIVED_CB2010","EJSCREEN_FLAG_US"]
for column in ej_columns:
  print_num_missing_per_column(column)

# Data Errors

In this section, we'll explore some of the obvious data errors in this database, and run a verifier that checks for duplicate and missing IDs.

## Some facilities have latitudes and longitudes that are outside the US
The following code and image depict the minimum and maximum latitudes and longtitudes within the dataset. As evidenced in the image, the minimum and maximum values lie outside of the boundary of the United States and territories as the highlighted square is the boundary created by these values.

In [ ]:
print("Minimum longitude:", full_echo_data.FAC_LONG.min())
print("Maximum longitude:", full_echo_data.FAC_LONG.max())
print("Minimum latitude:", full_echo_data.FAC_LAT.min())
print("Maximum latitude:",full_echo_data.FAC_LAT.max())

![picture](https://github.com/tzou2024/PInTxEdgi/blob/main/bodningbox.png?raw=true)

## Some facilities have unusual state abbreviations in `FAC_STATE`

The state values in `FAC_STATE` should match known state or territorial postal codes. This function loads all of the unique values in `FAC_STATE` and compares them against a list of state and territorial postal codes. The values that don't match the postal codes, and the number of facilities using those values, are put in `results`. Some of these incorrect codes include Canadian province codes, like PQ, AB, and BC. Others, like XF, refer to broad areas like "Americas." GE refers to Georgia the country [according](https://support.isan.org/hc/en-us/articles/360012636280-List-of-ISO-3166-Country-Codes) to the ISO. 

Note that areas such as Washington DC, Puerto Rico, and Guam are acceptable state postal codes.

In [ ]:
f = open('ECHO_modules/state_postal_code_to_fips.json',)
state_postal_code_to_fips = json.load(f)

results = {
    "Nonexistent State Abbreviation": [],
    "# Facilities With Abbreviation": []
}

state_values = full_echo_data[["FAC_STATE"]].values.ravel()
unique_state_values =  pd.unique(state_values)
total_incorrect = 0;

for value in unique_state_values: 
  if value not in state_postal_code_to_fips and isinstance(value, str):
    count = full_echo_data[full_echo_data["FAC_STATE"] == value].size
    total_incorrect += count
    results["Nonexistent State Abbreviation"].append(value)
    results["# Facilities With Abbreviation"].append(count)
results = pd.DataFrame(results)
print("Total # Facilities With Nonexistent State Abbreviations:", total_incorrect)
results

## Facilities that have no IDs, even with the flag checked

When a facility is marked to report underneath a certain act, then the facility is given an ID that corresponds to that act. That ID can be used to find more about that facility's compliance with that act outside of this summary ECHO dataset.

For CWA, the corresponding ID is `NDPES_ID`. If a facility must report under the CWA, it should have the flag `NPDES_FLAG` checked as `Y`. However, if `NPDES_FLAG` is `Y` but the `NDPES_ID` is missing, we can't find out more information about this facility that isn't in the summary data set.

This cell checks if any facilities are marked for a given act but are missing the ID.

In [ ]:
# find facilities where flag is there but ids aren’t
print("CWA Facilities that are missing NDPES_IDS")
display(cwa_facs[cwa_facs['NPDES_IDS'].isnull()].shape[0])

print("RCRA Facilities that are missing RCRA_IDS")
display(rcra_facs[rcra_facs['RCRA_IDS'].isnull()].shape[0])

print("CAA Facilities that are missing AIR_IDS")
display(caa_facs[caa_facs['AIR_IDS'].isnull()].shape[0])

## Duplicate IDs

Registry ID, NDPES_IDS, RCRA_IDS, and AIR_IDS should all be unique values that each only occur once in the database. That is, each row (each facility) should have a different Registry ID. Likewise, each facility regulated under the CWA should have a different NPDES_ID. Among the values that are not null, let's check whether there are any duplicate values.

In [ ]:
print("All facilities with duplicate REGISTRY_IDS")
facs_with_registry_ids = full_echo_data[full_echo_data['REGISTRY_ID'].notnull()]
display(facs_with_registry_ids[facs_with_registry_ids.duplicated(['REGISTRY_ID'], keep=False)].shape[0])

print("CWA Facilities with duplicate NDPES_IDS")
cwa_facs_with_cwa_ids = cwa_facs[cwa_facs['NPDES_IDS'].notnull()]
display(cwa_facs_with_cwa_ids[cwa_facs_with_cwa_ids.duplicated(['NPDES_IDS'], keep=False)].shape[0])

print("RCRA Facilities with duplicate RCRA_IDS")
rcra_facs_w_rcra_ids = rcra_facs[rcra_facs["RCRA_IDS"].notnull()]
display(rcra_facs_w_rcra_ids[rcra_facs_w_rcra_ids.duplicated(['RCRA_IDS'], keep=False)].shape[0])

print("CAA Facilities with duplicate AIR_IDS")
caa_facs_w_caa_ids = caa_facs[caa_facs["AIR_IDS"].notnull()]
display(caa_facs_w_caa_ids[caa_facs_w_caa_ids.duplicated(['AIR_IDS'], keep=False)].shape[0])

# Calculating Scores for % of Missing Data
This section generates data quality scores for each facility that can be used to conduct region-level analyses.

An important caveat to note: it's possible that there are regulated facilities that aren't even in the database. This basic info score can't take into account those facilities.

This function generates each facility's score for a given act by computing the percent of columns related to that act that are missing data. Since this is a percentage of relevant columns that are missing data, **higher values mean more data is missing.**

* A score of **0** means that there is **no missing data**.
* A score of **100** means that **all relevant columns are missing data.**

Run this cell in order to prepare the analysis. It won't generate output other than a message that says "Ready".

In [ ]:
# function that calculates the amount of missing data for the act in the intended columns; computes score and assigns to intended column
def calc_score(intended_columns, act, type_of_score):
  def labeler (row):
    total_missing = 0
    for col in intended_columns:
      # for the columns that are strings, check if they are null
      if col == "FAC_STATE" or col == "FAC_COUNTY":
        if pd.isnull(row[col]):
          total_missing = total_missing + 1
      # otherwise, check if they are not a number
      elif math.isnan(row[col]):
        total_missing = total_missing + 1
    return round((total_missing / len(intended_columns) * 100), 2)
  act[type_of_score] = act.apply(labeler, axis=1)

print("Ready")

### Basic Info Score
Run the score calculator for basic information for each facility EPA has in its database, including information on registry ID, facility latitude and longitude, facility county, and facility state. The resulting table shows only the relevant columns, and the computed score.

This may take a minute to run since we are calculating scores for over 1 million facilities.

In [ ]:
calc_score(intended_columns["Basic Info"], full_echo_data, "Basic_Info_Score")
basic_info_columns_and_score = intended_columns["Basic Info"].copy()
basic_info_columns_and_score.append('Basic_Info_Score')
full_echo_data[basic_info_columns_and_score]

### CWA Score
Run the score calculator for CWA information on CWA-flagged facilities. The resulting table shows only the relevant columns, and the computed score.

In [ ]:
calc_score(intended_columns["CWA"], cwa_facs, "CWA_Score")
cwa_columns_and_score = intended_columns["CWA"].copy()
cwa_columns_and_score.append('CWA_Score')
cwa_facs[cwa_columns_and_score]

### RCRA Score
Run the score calculator for RCRA information on RCRA-flagged facilities. The resulting table shows only the relevant columns, and the computed score.

In [ ]:
calc_score(intended_columns["RCRA"], rcra_facs, "RCRA_Score")
rcra_columns_and_score = intended_columns["RCRA"].copy()
rcra_columns_and_score.append('RCRA_Score')
rcra_facs[rcra_columns_and_score]

### CAA Score
Run the score calculator for CAA information on CAA-flagged facilities. The resulting table shows only the relevant columns, and the computed score.

In [ ]:
calc_score(intended_columns["CAA"], caa_facs, "CAA_Score")
caa_columns_and_score = intended_columns["CAA"].copy()
caa_columns_and_score.append('CAA_Score')
caa_facs[caa_columns_and_score]

## Average facility scores by state
This function averages the scores for every facility in each state to get state-level scores for each act.

Run this cell to prepare the analysis. It won't generate output other than "Ready".

In [ ]:
# calculate score by state by averaging all facilities in that state
def calc_score_by_state(act, act_state, act_name):
  act_state = act.groupby(["FAC_STATE"], dropna=False).agg({act_name + "_Score": "sum", "FAC_STATE": "count"})

  act_state[act_name + "_Average"] = round((act_state[act_name + "_Score"] / act_state["FAC_STATE"]), 2)
  act_state["STUSPS"] = act_state.index

  # remove "inf" values resulting from no state listed
  act_state = act_state.drop(act_state[act_state[act_name + "_Average"]>100].index)
  act_state = act_state.drop(columns=["FAC_STATE",(act_name + "_Score")])
  return act_state

print("Ready")

Call the previous block's state averaging function for each score (basic info, CWA, RCRA, CAA), and merge all the scores together into one table.

Remember that lower scores indicate greater data completeness.

In [ ]:
# Merge on state for Basic Info score
basic_info_state_data = None
basic_info_state_data = calc_score_by_state(full_echo_data, basic_info_state_data, "Basic_Info")

# Merge on state for CWA score
cwa_state_data = None
cwa_state_data = calc_score_by_state(cwa_facs, cwa_state_data, "CWA")

# Merge on state for RCRA score
rcra_state_data = None
rcra_state_data = calc_score_by_state(rcra_facs, rcra_state_data, "RCRA")

# Merge on state for CAA score
caa_state_data = None
caa_state_data = calc_score_by_state(caa_facs, caa_state_data, "CAA")

# Merge all scores together
scores_by_state = reduce(lambda  left,right: pd.merge(left,right,on=['STUSPS'],
  how='outer'), [basic_info_state_data, cwa_state_data, rcra_state_data, caa_state_data])
scores_by_state

Our scoring function has captured the unusual state/territorial abbreviations like "XI", which significantly skews the data when trying to calculate standard deviations. Let's remove the rows in `scores_by_state` with the unusual state/territorial abbrevations.

In [ ]:
# Drop the states with nonexistent abbreviations
scores_by_state = scores_by_state[scores_by_state["STUSPS"].isin(state_postal_code_to_fips)]
scores_by_state

Now, let's calculate the mean-adjusted standard deviation for the basic info score. Because the basic info scores are small, the mean-adjusted standard deviation for the basic info will give us better information on how well states are doing relative to each other.

Results show the number of standard deviations the state or territory's basic info score is from the mean. In this case, negative scores show *above* average data completness, while positive scores indicate *below* average data completeness. (For reference, we are calculating [Z-scores](https://en.wikipedia.org/wiki/Standard_score))

In [ ]:
# Since the difference between basic_info scores is low (generally less than 1),
std = scores_by_state['Basic_Info_Average'].std() # calculate the standard deviation
mean = scores_by_state['Basic_Info_Average'].mean() # calculate overall mean
def stder(row):
  v = round(((row["Basic_Info_Average"] - mean) / std), 2) # calculate departure from mean in terms of standard deviations 
  return v
scores_by_state["Basic_Info_Average_STD"] = scores_by_state.apply(stder, axis=1) 

print("Standard Deviation: " + str(std))
print("Mean: " + str(mean))
scores_by_state

## Visualize State Scores With a Map

### Basic Info Score Standard Deviation Map
Keep in mind that a higher departure from the average basic info score is worse. Therefore,
* Red = worse data completeness
* White = better data completeness:

In [ ]:
state_choropleth_mapper(scores_by_state, "Basic_Info_Average_STD", "Departure from Average Basic Info Score by State")

### CWA Score Map
Keep in mind that higher scores are worse, as they indicate the average percent of missing data for CWA facilities in that state. 
* Red = worse data completeness
* White = better data completeness

In [ ]:
state_choropleth_mapper(scores_by_state, "CWA_Average", "CWA Score by State")

### RCRA Score Map
Keep in mind that higher scores are worse, as they indicate the average percent of missing data for RCRA facilities in that state. 
* Red = worse data completeness
* White = better data completeness

In [ ]:
state_choropleth_mapper(scores_by_state, "RCRA_Average", "RCRA Score by State")

### CAA Score Map
Keep in mind that higher scores are worse, as they indicate the average percent of missing CAA data for facilities in that state. 
* Red = worse data completeness
* White = better data completeness

In [ ]:
state_choropleth_mapper(scores_by_state, "CAA_Average", "CAA Score by State")

# Further explore missing EJ data



In EPA's dataset, there are two ways of determining if a facility is in an environmental justice community.

* `FAC_PERCENT_MINORITY` is the percentage of the population within a 3-mile radius that is minority. If this value is above 50%, then the facility is located in a majority-minority community.
* `EJSCREEN_FLAG_US` indicates whether a facility is located in a census block groups that is in the 80th or higher national percentile of one of the primary environmental justice indexes of EJSCREEN, EPA's screening tool for EJ concerns. If a facility is in the 80th percentile, then it will have an `EJSCREEN_FLAG_US` value of `Y`. 

In this notebook, we've provided two options to sort facilities into environmental justice communities. By default, we will use `FAC_PERCENT_MINORITY`, but if you'd like to explore the effects of `EJSCREEN_FLAG_US`, you can change the column here: 

In [ ]:
choose_ej_column_widget = widgets.RadioButtons(
    options=['FAC_PERCENT_MINORITY', 'EJSCREEN_FLAG_US'],
    value='FAC_PERCENT_MINORITY', # Default
    description='Column:',
    disabled=False
)
display(choose_ej_column_widget)

If `FAC_PERCENT_MINORITY` is selected, then:
* `majmin` contains facilities that are in communities that are more than 50% minority.
* `white` contains facilities that are in communities that are less than 50% minority.

If `EJSCREEN_FLAG_US` is selected, then:
* `majmin` contains facilities that are marked in the 80th percentile for EJScreen.
* `white` contains facilities that are **not** marked in the 80th percentile for EJScreen

# Comparing Scores for Majority Minority versus Majority White Communities

This function generates the total score for each act according to demographics by computing the percent of columns related to that act are missing. Since this is a percentage of columns that are missing, higher values mean more data is missing.

*   A score of 0 means that there is no missing data.
*   A score of 100 means that all relevant columns are missing data.

Run this cell to proceed with the analysis. It won't generate any output other than ("Ready")

In [ ]:
def calc_by_demographic(act, act_name):
  results = {}
  results[act_name] = {}
  results[act_name]['All'] = act[act_name + "_Score"].sum() / len(act)

  # the way maj-min is calculated changes based on the column widget (see above)
  # Majority-minority
  if choose_ej_column_widget.value == "FAC_PERCENT_MINORITY":  
    act_majmin_facs = act.loc[act["FAC_PERCENT_MINORITY"] >= 50]
    majmin_title = "Majority-Minority"

    act_majwhite_facs = act.loc[act["FAC_PERCENT_MINORITY"] < 50]
    majwhite_title = "Majority-White"
  else:
    act_majmin_facs = act[act["EJSCREEN_FLAG_US"] == "Y"]
    majmin_title = ">80th Percentile EJScreen"

    act_majwhite_facs = act[act["EJSCREEN_FLAG_US"] == "N"]
    majwhite_title = "<80th Percentile EJScreen"

  results[act_name][majmin_title] = round((act_majmin_facs[act_name + "_Score"].sum() / len(act_majmin_facs)),2)
  results[act_name][majwhite_title] = round((act_majwhite_facs[act_name + "_Score"].sum() / len(act_majwhite_facs)),2)
  # Calculate difference between majmin and majwhite
  results[act_name]['Difference'] = results[act_name][majmin_title] - results[act_name][majwhite_title]

  results = pd.DataFrame(results).transpose()
  return results

print("Ready")

This cell calculates the missing data scores by demographics for each act. The resulting scores are then combined into one table and displayed.

Positive values for `Difference` indicate that facilities in Majority-Minority neighborhoods (as defined by the column selected) have *less* data completeness than facilities in Majority-White neighborhoods.

In [ ]:
cwa_results = calc_by_demographic(cwa_facs, "CWA")
rcra_results = calc_by_demographic(rcra_facs, "RCRA")
caa_results = calc_by_demographic(caa_facs, "CAA")
scores_by_demographics = pd.concat([cwa_results, rcra_results], axis=0)
scores_by_demographics = pd.concat([scores_by_demographics, caa_results], axis=0)
scores_by_demographics

Here we produce a visualization of the data.

In [ ]:
scores_by_demographics.drop(columns="Difference").plot.bar(figsize=(12,12),ylim=(0,100)).set(ylabel = '% Facilities Missing Data in Column', title = 'Missing Facility Data for All vs. Majority-Minority vs. White Communities')

# Visualize missing data for majority-minority vs. maority-white communities for a specific act
Let's compare the data completeness for majority-minority vs. white communities for a specific act. Run this cell to choose a specific act:

In [ ]:
choose_act_widget=widgets.Select(
    options=["Basic Info", "CWA", "RCRA", "CAA"],
    description='Act:',
    disabled=False
)
display(choose_act_widget)

Similar to the way that we previously split facilities into environmental justice and non-environmental justice communities, here we'll split facilities based on the environmental justice column selection from above.

In [ ]:
# Separate into majority nonwhite and majority white areas
# based on the column value selected earlier
currently_selected_facs = act_facs[choose_act_widget.value]
if choose_ej_column_widget.value == "FAC_PERCENT_MINORITY":  
  majmin = currently_selected_facs.loc[currently_selected_facs["FAC_PERCENT_MINORITY"] >= 50 ] # 40 = US-wide percent
  white = currently_selected_facs.loc[currently_selected_facs["FAC_PERCENT_MINORITY"] < 50 ]
else:
  majmin = currently_selected_facs[currently_selected_facs["EJSCREEN_FLAG_US"] == "Y"]
  white = currently_selected_facs[currently_selected_facs["EJSCREEN_FLAG_US"] == "N" ]

print("% Facilities in majority minority areas")
print(round((len(majmin.index)/(len(majmin.index) + len(white.index))*100),2), "%")
print("--------------")
print("% Facilities in majority white areas")
print(round((len(white.index)/(len(majmin.index) + len(white.index)) * 100),2), "%")

### Get percent of missing data for each column in this act

This cell gets the percent of missing data for each column in this act, then compares it for facilities in majority-minority communities vs. majority-white communities.

As this is the percent of missing data, **higher scores are worse**, because it means more data is missing.


---

Note: when looking at Basic Info, the data completeness score will always be 0 for `FAC_LONG` and `FAC_LAT`. If the ECHO database doesn't have a facility's location, then it also won't have the minority composition of the surrounding community. Therefore, none of the facilities that are missing `FAC_LONG` and `FAC_LAT` will be sorted into `majmin` or `white`.

In [ ]:
results = {}
chosen_act_columns = intended_columns[choose_act_widget.value]

groups = ["All", "Majority-Minority", "Majority-White"]
for pos, group in enumerate([currently_selected_facs, majmin, white]):
  results[groups[pos]]={}
  facilities = len(group.index)
  for field in chosen_act_columns:
    results[groups[pos]]["% Facilities Missing "+field] = round(((group[field].isna().sum()/facilities)*100),2)
results = pd.DataFrame(results)
results

The following cell visualizes the previous table into a graph.

In [ ]:
results.plot.bar(figsize=(12,12)).set(ylabel = '% Facilities Missing Data in Column', title = 'Missing Facility Data for All vs. Majority-Minority vs. White Communities');

# Zooming in on data disparities in specific columns
Choose a column to calculate the data quality - specifically, the percent of facilities in majority-minority communities regulated under the selected act with data missing in this column, per state. If you would like to switch acts, return to the `choose_act_widget` a few cells above and then re-run them.

In [ ]:
choose_column_widget=widgets.Select(
    options=chosen_act_columns,
    description='Column:',
    disabled=False
)
display(choose_column_widget)

The cell below calculates the percent of missing data in majority minority communities by state.

* `Count`: number of facilities regulated under this act in majority minority communities with missing information in the selected column, per state
* `Percent`: the above, expressed as a percent of all 
* `STD`: the standard deviation, indicates if that state has more or less facilities with missing information compared to the mean number of facilities with missing information. Here, negative values indicate *greater* data completeness compared with the US mean, while positive values indicate *less* data completeness.

In plain language, a result like
| FAC_STATE  | Count  | % Missing Column  | STD    |
| ---------- | ------ | ----------------- | ------ |
| AK         | 139    | 43.99             | -0.49  |

means that there are 139 facilities regulated under the selected act in Alaska that are in majority-minority areas and are missing information on the selected column. This equates to 43.99% of all facilities regulated under the selected act in Alaskan majority-minority communities. This represents a somewhat greater level of data completness in Alaska's majority-minority communities than other states.

In [ ]:
chosen_column = choose_column_widget.value

#add column for counting NAs on X column (e.g. CWA_INSPECTION_COUNT)
def labeler (row):
  if chosen_column == "FAC_STATE" or chosen_column == "FAC_COUNTY":
    if pd.isnull(row[chosen_column]):
      return 1
    else:
      return 0
  else:
    if math.isnan(row[chosen_column]):
      return 1
    else:
      return 0
majmin["Count"] = majmin.apply(labeler, axis=1) # count NAs on X column

# Summarize the # of facilities in maj-min communities missing info per state 
# and also just count the total number of facilities in maj-min communities per state
state_data = majmin.groupby(["FAC_STATE"], dropna=False).agg({"Count": "sum", "FAC_STATE": "count"}) 
# Calculate the previous as a percentage
state_data["% Missing Column"] = round(((state_data["Count"] / state_data["FAC_STATE"]) * 100),2)
state_data["STUSPS"] = state_data.index

# Remove "inf" values resulting from no state listed
state_data = state_data.drop(state_data[state_data["% Missing Column"]>100].index)

mean = state_data['% Missing Column'].mean() # Calculate average across states
std = state_data['% Missing Column'].std() # Calculate standard deviation
def stder(row):
  z = round(((row["% Missing Column"]-mean)/std),2)
  return z
state_data["STD"] = state_data.apply(stder, axis=1) # calculate departure from mean in terms of STD (Z scores)

display(state_data[["Count", "% Missing Column", "STD"]])


The map below shows the variation in the average percent of facilities in majority minority communities regulated under the selected act with no information for the selected column.

The map is generated by getting geographic information on each of the states, and then matching this geographic information with the state-level standard deviation previously calculated.

* Red = worse data completeness
* White = better data completeness
* Black indicates no data on that state.

In [ ]:
state_choropleth_mapper(state_data, "STD", "Departure from Average % of Facilities Missing " + chosen_column)

# What does missing data look like where I live?

This displays the scores for your state, and which columns are missing the most data in your county.

Note: since this uses the `FAC_STATE` and `FAC_COUNTY` columns to find facilities in your area, this won't capture facilities in your county that are missing state and county data.

Pick your state:

In [ ]:
from ECHO_modules.utilities import show_region_type_widget, \
    show_state_widget, show_pick_region_widget, get_active_facilities
state_widget = show_state_widget()

Pick one county:

In [ ]:
region_widget = show_pick_region_widget( type='County',
  state_widget=state_widget )

This cell parses the outputs of the widgets into a value that can be used to run queries. Please run it, but the only ouput you'll see is "Ready".

In [ ]:
state = state_widget.value

if region_widget.value:
  regions_selected = region_widget.value
  parseable_county_name = "','".join( region_widget.value )
else: # set default
  regions_selected = ('BALDWIN',) # random county in Alabama

print("Ready")

The following is the average Basic Info, CWA, RCRA, and CAA for the selected state:

In [ ]:
scores_by_state.loc[scores_by_state["STUSPS"] == state]

The following is the average Basic Info, CWA, RCRA, and CAA for the selected county.

Remember that in this case, higher scores indicate *less* data completeness, while lower scores indicate *greater* data completness.

In [ ]:
county_facs = full_echo_data[full_echo_data["FAC_COUNTY"] == parseable_county_name]
print("Basic Info Score for your county")
display(round((county_facs.sum().Basic_Info_Score / len(county_facs)),2))
print("-------------------------")

cwa_county_facs = cwa_facs[cwa_facs["FAC_COUNTY"] == parseable_county_name]
print("CWA Score for your county")
display(round((cwa_county_facs.sum().CWA_Score / len(cwa_county_facs)),2))
print("-------------------------")

rcra_county_facs = rcra_facs[rcra_facs["FAC_COUNTY"] == parseable_county_name]
print("RCRA Score for your county")
display(round((rcra_county_facs.sum().RCRA_Score / len(rcra_county_facs)),2))
print("-------------------------")

caa_county_facs = caa_facs[caa_facs["FAC_COUNTY"] == parseable_county_name]
print("CAA Score for your county")
display(round((caa_county_facs.sum().CAA_Score / len(caa_county_facs)),2))

**!!! HERE BE DRAGONS: The following code block needs testing and validation.**

The following is the top 20 columns with missing data in the selected county.

In [ ]:
selected_region_facilities = get_active_facilities( state, 'County', regions_selected )

# if you only want to get the columns from full_echo_data
# selected_region_facilities = full_echo_data.loc[full_echo_data['FAC_COUNTY'] == regions_selected]

num_facs = selected_region_facilities.shape[0]
missing_columns = selected_region_facilities.isnull().sum().to_frame()
missing_columns['%'] = missing_columns[0] / num_facs

max_columns_to_show = 20
# sort columns by highest percentage of missing data
missing_columns = missing_columns.sort_values(by=['%'], ascending=False)
res = missing_columns.head(max_columns_to_show)
print("Top 20 columns with missing data in", parseable_county_name, state)
res